<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Strasbourg

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Strasbourg_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Strasbourg - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1

In [7]:
class StrasbourgSpider(scrapy.Spider) :
    # Name of spider
    name = 'Strasbourg'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=strasbourg&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=strasbourg&search_pageview_id=15c43bf6873c02b4&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(StrasbourgSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
strasbourg = pd.read_json('results_hotels/Strasbourg_multipage.json')

In [11]:
len(strasbourg)

125

In [12]:
strasbourg.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/hotel-de-roha...,Hotel Rohan,"8,6",L'Hôtel De Rohan est situé dans le centre hist...
1,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/chut.fr.html?...,Hôtel Restaurant Le Chut - Petite France,"8,3",L'hôtel Chut est un hôtel agréable au style co...
2,"Bourse-Esplanade, Strasbourg",https://www.booking.com/hotel/fr/appartement-d...,L'Appartement de l'Observatoire,"9,2",Hébergement géré par un particulier
3,"Gare de Strasbourg, Strasbourg",https://www.booking.com/hotel/fr/ibis-strasbou...,ibis Strasbourg Centre Petite France,"7,7",L'hôtel ibis Strasbourg Centre Petite France v...
4,Strasbourg,https://www.booking.com/hotel/fr/appart-hote-r...,Résidence les Cèdres,"8,2",La Résidence les Cèdres se trouve à Strasbourg...
5,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/les-apparteme...,Les Appartements de l'Epine,"9,1",Hébergement géré par un particulier
6,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/strasbourg-ap...,Strasbourg Appart Rue du 22 Novembre,"8,0","Situé à Strasbourg, à 2,1 km de la Petite Fran..."
7,"Quartier Européen - Orangerie, Strasbourg",https://www.booking.com/hotel/fr/de-l-orangeri...,Hotel de l'Orangerie,"7,8",L'Hôtel de l'Orangerie est situé dans le quart...
8,Strasbourg,https://www.booking.com/hotel/fr/strasbourg-lo...,Strasbourg Urban Chic Cathedral View - parking...,"9,0",Hébergement géré par un particulier
9,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/leonor-the-pl...,Hôtel LÉONOR the place to live,"9,2","Situé à Strasbourg, à 200 mètres du marché de ..."


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in strasbourg['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
strasbourg['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in strasbourg['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
strasbourg['latitude'] = lat
strasbourg['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in strasbourg['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
strasbourg['hotel_description_long'] = result_long_description

In [20]:
strasbourg.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/hotel-de-roha...,Hotel Rohan,"8,6",L'Hôtel De Rohan est situé dans le centre hist...,"48.58094495,7.75043070",48.58094495,7.75043070,\nL'Hôtel De Rohan est situé dans le centre hi...
1,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/chut.fr.html?...,Hôtel Restaurant Le Chut - Petite France,"8,3",L'hôtel Chut est un hôtel agréable au style co...,"48.58126258,7.73946315",48.58126258,7.73946315,\nVous pouvez bénéficier d'une réduction Geniu...
2,"Bourse-Esplanade, Strasbourg",https://www.booking.com/hotel/fr/appartement-d...,L'Appartement de l'Observatoire,"9,2",Hébergement géré par un particulier,"48.58076480,7.76955290",48.58076480,7.76955290,\nVous pouvez bénéficier d'une réduction Geniu...
3,"Gare de Strasbourg, Strasbourg",https://www.booking.com/hotel/fr/ibis-strasbou...,ibis Strasbourg Centre Petite France,"7,7",L'hôtel ibis Strasbourg Centre Petite France v...,"48.58253971,7.73732672",48.58253971,7.73732672,\nL'hôtel ibis Strasbourg Centre Petite France...
4,Strasbourg,https://www.booking.com/hotel/fr/appart-hote-r...,Résidence les Cèdres,"8,2",La Résidence les Cèdres se trouve à Strasbourg...,"48.59089809,7.71870852",48.59089809,7.71870852,\nVous pouvez bénéficier d'une réduction Geniu...
5,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/les-apparteme...,Les Appartements de l'Epine,"9,1",Hébergement géré par un particulier,"48.58013240,7.74849210",48.58013240,7.74849210,\nVous pouvez bénéficier d'une réduction Geniu...
6,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/strasbourg-ap...,Strasbourg Appart Rue du 22 Novembre,"8,0","Situé à Strasbourg, à 2,1 km de la Petite Fran...","48.58254364,7.74379355",48.58254364,7.74379355,\nVous pouvez bénéficier d'une réduction Geniu...
7,"Quartier Européen - Orangerie, Strasbourg",https://www.booking.com/hotel/fr/de-l-orangeri...,Hotel de l'Orangerie,"7,8",L'Hôtel de l'Orangerie est situé dans le quart...,"48.59048760,7.76852780",48.59048760,7.76852780,\nL'Hôtel de l'Orangerie est situé dans le qua...
8,Strasbourg,https://www.booking.com/hotel/fr/strasbourg-lo...,Strasbourg Urban Chic Cathedral View - parking...,"9,0",Hébergement géré par un particulier,"48.58059820,7.71662750",48.58059820,7.71662750,\nVous pouvez bénéficier d'une réduction Geniu...
9,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/leonor-the-pl...,Hôtel LÉONOR the place to live,"9,2","Situé à Strasbourg, à 200 mètres du marché de ...","48.58594200,7.74723500",48.58594200,7.74723500,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
strasbourg['hotel_description_long'] = strasbourg['hotel_description_long'].replace('\n','', regex=True)

In [22]:
strasbourg.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/hotel-de-roha...,Hotel Rohan,"8,6",L'Hôtel De Rohan est situé dans le centre hist...,"48.58094495,7.75043070",48.58094495,7.75043070,L'Hôtel De Rohan est situé dans le centre hist...
1,Centre de Strasbourg - Petite France - Cathédr...,https://www.booking.com/hotel/fr/chut.fr.html?...,Hôtel Restaurant Le Chut - Petite France,"8,3",L'hôtel Chut est un hôtel agréable au style co...,"48.58126258,7.73946315",48.58126258,7.73946315,Vous pouvez bénéficier d'une réduction Genius ...
2,"Bourse-Esplanade, Strasbourg",https://www.booking.com/hotel/fr/appartement-d...,L'Appartement de l'Observatoire,"9,2",Hébergement géré par un particulier,"48.58076480,7.76955290",48.58076480,7.76955290,Vous pouvez bénéficier d'une réduction Genius ...
3,"Gare de Strasbourg, Strasbourg",https://www.booking.com/hotel/fr/ibis-strasbou...,ibis Strasbourg Centre Petite France,"7,7",L'hôtel ibis Strasbourg Centre Petite France v...,"48.58253971,7.73732672",48.58253971,7.73732672,L'hôtel ibis Strasbourg Centre Petite France v...
4,Strasbourg,https://www.booking.com/hotel/fr/appart-hote-r...,Résidence les Cèdres,"8,2",La Résidence les Cèdres se trouve à Strasbourg...,"48.59089809,7.71870852",48.59089809,7.71870852,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
strasbourg['hotel_description_long'][55]

'Situé dans la zone piétonne près de la place Kléber, l’établissement historique Maison Rouge est agrémenté de vitraux d’époque. Ses logements sont décorés dans un style personnalisé. Une connexion Wi-Fi gratuite est disponible dans l’ensemble des locaux.Les chambres du Maison Rouge sont accessibles aux personnes à mobilité réduite et comprennent un minibar ainsi qu’une télévision par satellite. Tous les hébergements sont desservis par l’ascenseur traditionnel de l’hôtel et possèdent une salle de bains privative pourvue d’une baignoire ou d’une douche.Lors de votre séjour, vous pourrez lire la presse quotidienne et profiter d’une réception ouverte 24h/24. La ligne de tramway local, dont l’arrêt se trouve devant l’hôtel, dessert les sites historiques de Strasbourg, notamment la rue de la Mésange, accessible en 5 minutes. '

In [24]:
# Clean location column
strasbourg['location'] = ["Strasbourg" if value!="Strasbourg" else value for value in strasbourg["location"]]

In [25]:
strasbourg.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Strasbourg,https://www.booking.com/hotel/fr/hotel-de-roha...,Hotel Rohan,"8,6",L'Hôtel De Rohan est situé dans le centre hist...,"48.58094495,7.75043070",48.58094495,7.75043070,L'Hôtel De Rohan est situé dans le centre hist...
1,Strasbourg,https://www.booking.com/hotel/fr/chut.fr.html?...,Hôtel Restaurant Le Chut - Petite France,"8,3",L'hôtel Chut est un hôtel agréable au style co...,"48.58126258,7.73946315",48.58126258,7.73946315,Vous pouvez bénéficier d'une réduction Genius ...
2,Strasbourg,https://www.booking.com/hotel/fr/appartement-d...,L'Appartement de l'Observatoire,"9,2",Hébergement géré par un particulier,"48.58076480,7.76955290",48.58076480,7.76955290,Vous pouvez bénéficier d'une réduction Genius ...
3,Strasbourg,https://www.booking.com/hotel/fr/ibis-strasbou...,ibis Strasbourg Centre Petite France,"7,7",L'hôtel ibis Strasbourg Centre Petite France v...,"48.58253971,7.73732672",48.58253971,7.73732672,L'hôtel ibis Strasbourg Centre Petite France v...
4,Strasbourg,https://www.booking.com/hotel/fr/appart-hote-r...,Résidence les Cèdres,"8,2",La Résidence les Cèdres se trouve à Strasbourg...,"48.59089809,7.71870852",48.59089809,7.71870852,Vous pouvez bénéficier d'une réduction Genius ...


In [26]:
strasbourg.to_csv("Strasbourg.csv")